In [21]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')
import os
import pandas as pd
import numpy as np
# import seaborn as sns
from utilities import* 
from pandas import read_excel
from matplotlib import pyplot as plt
plt.rc('pdf',fonttype = 42)
plt.rc('ps',fonttype = 42)
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage,AnchoredOffsetbox
from matplotlib import rc, rcParams
rc('font', weight='bold')
import pymongo
from pymongo import MongoClient
from pymongo import InsertOne, DeleteMany, ReplaceOne, UpdateOne
from os import listdir
from os.path import isfile, join
import datetime
from pprint import pprint

In [26]:
def create_algo_detail(db):
    LaCAM = {
        "algo_name": "LaCAM",
        "authors": "Keisuke Okumura",
        "papers": "Keisuke Okumura. LaCAM: Search-Based Algorithm for Quick Multi-Agent Pathfinding. In Proceedings of the AAAI Conference on Artificial Intelligence, 2023",
        "github": "https://github.com/Kei18",
        "comments": "",
        'user_id':""
    }
    db['algorithms'].insert_one(LaCAM)
    
    
def get_path(filename,number_of_agents):
    with open(filename) as file:
        planString = ""
        number_of_line = 0
        for line in file:
            number_of_line = number_of_line + 1
            planString  = planString + line
        planString = planString.rstrip("\n")
        if(number_of_agents != number_of_line):
            print("Agents number not matched")
    return planString



def insert_submission(db,submission,instance_id,algo_name,path, trival_lower_bound):
    current_instance = db['instances'].find_one({'_id':instance_id})
    path_id = None
    if(submission['solution_cost']!= None):
        if(current_instance['lower_cost'] != None):
            if(current_instance['lower_cost'] > submission['solution_cost']):
                print("Inserting error lower larger than solution")
#         //insert solution
        if(current_instance['solution_cost'] == None):
            submission["best_solution"]= True
            db['instances'].update_one({"_id" : instance_id},{"$set": 
                                                              {"solution_cost": submission['solution_cost'],
                                                                "solution_date":submission['date'],
                                                                "solution_path": path,
                                                                "empty": False
                                                              },
                                                              "$push":
                                                              {
                                                                 "solution_algos":{
                                                                     "algo_name": algo_name,
                                                                     "algo_id": submission['algo_id'],
                                                                     "date":submission['date'],
                                                                 }
                                                              }
                                                             })
        else:
            if(current_instance['solution_cost'] > submission['solution_cost']):
                db['submissions'].update_many({"instance_id" :  instance_id},{"$set": 
                                                      { 
                                                          "best_solution" : False
                                                      }
                                                     })
                submission["best_solution"]= True
                db['instances'].update_one({"_id" : instance_id},{"$set": 
                                                                    {
                                                                "solution_cost": submission['solution_cost'],
                                                                "solution_algos":[{
                                                                     "algo_name": algo_name,
                                                                     "algo_id": submission['algo_id'],
                                                                     "date":submission['date'],
                                                                    }],
                                                                "solution_date": submission['date'],
                                                                "solution_path": path,
                                                                "empty": False
                                                                    }
                                                             })
                
            elif (current_instance['solution_cost'] == submission['solution_cost']):
                submission["best_solution"]= True
                db['instances'].update_one({"_id" : instance_id},{
                                                  "$push":
                                                  {
                                                     "solution_algos":{
                                                         "algo_name": algo_name,
                                                         "algo_id": submission['algo_id'],
                                                         "date":submission['date'],
                                                     }
                                                  }
                                                 })   
          
    if(current_instance['lower_cost'] == None):
        db['instances'].update_one({"_id" : instance_id},{"$set": 
                                                  {"lower_cost": trival_lower_bound,
                                                    "lower_date":submission['date'],
                                                    "empty": False
                                                  }
                                                 })  
        
    current_instance = db['instances'].find_one({'_id':instance_id})
    if current_instance['lower_cost'] == current_instance['solution_cost']:
        db['instances'].update_one({"_id" : instance_id},{"$set": 
                          {"closed": True}
                         })  
    db['submissions'].insert_one(submission)
        
def insert_lower_only(db,instance_id, date, trival_lower_bound):
    current_instance = db['instances'].find_one({'_id':instance_id})
    if(current_instance['lower_cost'] == None):
        db['instances'].update_one({"_id" : instance_id},{"$set": 
                                                  {"lower_cost": trival_lower_bound,
                                                    "lower_date":date,
                                                    "empty": False
                                                  }
                                                 })    
        
        
def insert_map_results(db,map_name):
    algo_id = db['algorithms'].find_one({"algo_name":"LaCAM"})['_id']      
    algo_name = "LaCAM"
    scen_type = ["even","random"]
    map_id = db["maps"].find_one({"map_name": map_name})['_id']
    for i in range(1, 26):
        for scen_t in scen_type: 
            result_file = "/home/ubuntu/mounted/lacam_experiments/lacam/results/"+map_name+"/statistic/"+map_name+"-"+scen_t+"-"+str(i)+"-60"
            print(result_file)
            df1 = load_data(result_file)
            scen_data =  db['scenarios'].find_one({"map_id": map_id ,"type_id": i, "scen_type" :  scen_t})
            scen_id = scen_data['_id']
            number_agent = scen_data['instances']
            for index, row in df1.iterrows(): 
                if(row['#agents'] > number_agent):
                    break;
                instance_id = db['instances'].find_one({"scen_id": scen_id ,"agents": row['#agents']})['_id']
                if row['solution cost'] != -1:
                    path_results =  "/home/ubuntu/mounted/lacam_experiments/lacam/results/"+map_name+"/path/"+map_name+"-"+scen_t+"-"+str(i)+"-60/"+str(row['#agents']) +"-path.txt"
                    submission= {
                        "map_id": map_id,
                        "instance_id" :  instance_id,
                        "algo_id": algo_id,
                        "lower_cost" : None,
                        "solution_cost": row['solution cost'],
                        "date": str(datetime.datetime.now().date()),
                        "best_lower": False,
                        "best_solution": False
                    }
                    path = get_path(path_results,row['#agents'])
                    insert_submission(db,submission,instance_id,algo_name,path,row['soc_lb'])
                else:
                    if row['soc_lb'] > -1:
                        insert_lower_only(db,instance_id,str(datetime.datetime.now().date()),row['soc_lb'])

def update_map_progress(db, map_name):
    map_id = db["maps"].find_one({"map_name": map_name})['_id']
    scen_type = ["even","random"]
    total_s = 0
    total_l = 0
    for i in range(1, 26):
        for scen_t in scen_type: 
            scen_id = db['scenarios'].find_one({"map_id": map_id ,"type_id": i, "scen_type" :  scen_t})['_id']
        
            num_s = db["instances"].count_documents({"scen_id" : scen_id, "closed" : True})
            num_l = db["instances"].count_documents({"scen_id" : scen_id, "solution_cost" : {"$ne":None}})
            total_s = total_s + num_s
            total_l = total_l + num_l
            db['scenarios'].update_one({"_id" : scen_id}, {"$set":{
                "instances_closed" : num_s,
                "instances_solved" : num_l
            }})
    db['maps'].update_one({"_id" : map_id}, {"$set":{
                "instances_closed" : total_s,
                "instances_solved" : total_l
            }})
    
    
def upload_map_results(db, map_name):
    insert_map_results(db,map_name)
    update_map_progress(db, map_name)   

In [27]:
url = "mongodb://localhost:27017/"
client = pymongo.MongoClient(url)
db = client['MAPF_V2']
print("Connect successfully")
upload_map_results(db,"orz900d")
# finished_map=[""]
# create_algo_detail(db)
# folder = 'results'

# sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]
# print(sub_folders)
# for map_name in sub_folders:
#     print(map_name)
#     if map_name in finished_map:
#         print("Skip finished map: " + map_name)
#     else:
#         upload_map_results(db,map_name)
client.close()

Connect successfully
/home/ubuntu/mounted/lacam_experiments/lacam/results/orz900d/statistic/orz900d-even-1-60
/home/ubuntu/mounted/lacam_experiments/lacam/results/orz900d/statistic/orz900d-random-1-60
/home/ubuntu/mounted/lacam_experiments/lacam/results/orz900d/statistic/orz900d-even-2-60
/home/ubuntu/mounted/lacam_experiments/lacam/results/orz900d/statistic/orz900d-random-2-60
/home/ubuntu/mounted/lacam_experiments/lacam/results/orz900d/statistic/orz900d-even-3-60
/home/ubuntu/mounted/lacam_experiments/lacam/results/orz900d/statistic/orz900d-random-3-60
/home/ubuntu/mounted/lacam_experiments/lacam/results/orz900d/statistic/orz900d-even-4-60
/home/ubuntu/mounted/lacam_experiments/lacam/results/orz900d/statistic/orz900d-random-4-60
/home/ubuntu/mounted/lacam_experiments/lacam/results/orz900d/statistic/orz900d-even-5-60
/home/ubuntu/mounted/lacam_experiments/lacam/results/orz900d/statistic/orz900d-random-5-60
/home/ubuntu/mounted/lacam_experiments/lacam/results/orz900d/statistic/orz900d-